# url 정보를 따라 50개 페이지 각각의 가격과 주소를 가져와야한다.

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

import pandas as pd

In [40]:
df = pd.read_csv('../data_web_data/03. best_sandwiches_list_chicago_user.csv', index_col=0)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [41]:
print(df['URL'][0])

http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/


In [8]:
#post-11773 > div.grid-container.grid-parent.article-wrap > div.grid-12.prefix-1.tablet-grid-14.tablet-prefix-0.suffix-1.mobile-grid-100.mobile-prefix-0.mobile-suffix-0 > div.article-body > p.addy > em
from urllib.request import Request
req = Request(df['URL'][0], headers={'User-Agent':'Chrome'})
html = urlopen(req).read()

soup_tmp = BeautifulSoup(html, 'html.parser')

In [11]:
print(soup_tmp.find('p','addy'))

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>


In [12]:
price_tmp = soup_tmp.find('p', 'addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [13]:
import re
re.split('.,', price_tmp)

['\n$10. 2109 W. Chicago Ave', ' 773-772-040', ' theoldoaktap.com']

In [14]:
price_tmp = re.split('.,', price_tmp)[0]
price_tmp

'\n$10. 2109 W. Chicago Ave'

In [18]:
re.search('\$\d+\.(\d+)?', price_tmp).group()       # 가격 조회

'$10.'

In [19]:
tmp = re.search('\$\d+\.(\d+)?', price_tmp).group()
price_tmp[len(tmp) + 2:]        # 가격이 끝나는 지점의 위치를 이용해서 그 뒤는 주소로 생각

'2109 W. Chicago Ave'

---

In [26]:
price = []
address = []

for n in df.index[:3]:
    req = Request(df['URL'][n], headers={'User-Agent':'Chrome'})
    html = urlopen(req).read()

    soup_tmp = BeautifulSoup(html,'html.parser')
    gettings = soup_tmp.find('p', 'addy').get_text()

    price_tmp = re.split('.,', gettings)[0]
    tmp = re.search('\$\d+\.(\d+)?', price_tmp).group()

    price.append(tmp)
    address.append(price_tmp[len(tmp) + 2:])

    print(n)

0
1
2


In [27]:
price, address

(['$10.', '$9.', '$9.50'],
 ['2109 W. Chicago Ave', '800 W. Randolph St', ' 445 N. Clark St'])

In [28]:
price = []
address = []

for idx, rows in df[:3].iterrows():
    req = Request(rows['URL'], headers={'User-Agent':'Chrome'})
    html = urlopen(req).read()

    soup_tmp = BeautifulSoup(html,'html.parser')        # lxml을 사용하게 될수도 있다.
    gettings = soup_tmp.find('p', 'addy').get_text()

    price_tmp = re.split('.,', gettings)[0]
    tmp = re.search('\$\d+\.(\d+)?', price_tmp).group()

    price.append(tmp)
    address.append(price_tmp[len(tmp) + 2:])

    print(idx)

0
1
2


In [29]:
price, address


(['$10.', '$9.', '$9.50'],
 ['2109 W. Chicago Ave', '800 W. Randolph St', ' 445 N. Clark St'])

In [31]:
from tqdm import tqdm

price = []
address = []

for idx, row in tqdm(df.iterrows()):
    # html = urlopen(row['URL'])
    req = Request(row['URL'], headers={'User-Agent':'Chrome'})
    html = urlopen(req).read()

    soup_tmp = BeautifulSoup(html, 'html.parser')

    gettings = soup_tmp.find('p','addy').get_text()

    price_tmp = re.split('.,', gettings)[0]
    tmp = re.search('\$\d+\.(\d+)?', price_tmp).group()

    price.append(tmp)
    address.append(price_tmp[len(tmp) + 2 :])


50it [02:30,  3.01s/it]


In [33]:
len(price), len(address)

(50, 50)

In [34]:
df['Price'] = price
df['Address'] = address

In [35]:
df.head()

,Rank,Cafe,Menu,URL,Price,Address
0,1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10.,2109 W. Chicago Ave
1,2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.,800 W. Randolph St
2,3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50,445 N. Clark St
3,4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40,914 Noyes St
4,5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10.,825 W. Fulton Mkt


In [36]:
df = df.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df.set_index('Rank', inplace=True)
df.head()

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10.,2109 W. Chicago Ave
2,Au Cheval,Fried Bologna,$9.,800 W. Randolph St
3,Xoco,Woodland Mushroom,$9.50,445 N. Clark St
4,Al’s Deli,Roast Beef,$9.40,914 Noyes St
5,Publican Quality Meats,PB&L,$10.,825 W. Fulton Mkt


In [37]:
df.to_csv('../data_web_data/03. best_sandwiches_list_chicago_user2.csv', sep=',', encoding='utf-8')

In [38]:
df

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10.,2109 W. Chicago Ave
2,Au Cheval,Fried Bologna,$9.,800 W. Randolph St
3,Xoco,Woodland Mushroom,$9.50,445 N. Clark St
4,Al’s Deli,Roast Beef,$9.40,914 Noyes St
5,Publican Quality Meats,PB&L,$10.,825 W. Fulton Mkt
6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,$7.25,100 E. Walton St
7,Acadia,Lobster Roll,$16.,1639 S. Wabash Ave
8,Birchwood Kitchen,Smoked Salmon Salad,$10.,2211 W. North Ave
9,Cemitas Puebla,Atomica Cemitas,$9.,3619 W. North Ave
